In [1]:
import tempfile

import h5py
import imageio
import numpy as np
import os
import pandas as pd
import cv2

from PIL import Image

# access individual dictionary keys and values and store in viewable files
    # dictionary images (np arrays) get converted into videos
        # video 1: "image"
        # video 2: "other" "hand_image"
    # rest of dictionary gets converted into csv files
        # file 1: "robot_state"
        # file 2: "controller_info" and "task"
        # file 4: "action"

# hdf5 structure:
# HDF5 "outputs/session_0/episode_0.h5" {
# GROUP "/" {
# DATASET "action" {
#     DATATYPE  H5T_IEEE_F64LE
#     DATASPACE  SIMPLE { ( 111, 7 ) / ( H5S_UNLIMITED, 7 ) }
# }
# GROUP "controller_info" {
#     DATASET "controller_on" {
#         DATATYPE  H5T_ENUM {
#             H5T_STD_I8LE;
#             "FALSE"            0;
#             "TRUE"             1;
#         }
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "failure" {
#         DATATYPE  H5T_ENUM {
#             H5T_STD_I8LE;
#             "FALSE"            0;
#             "TRUE"             1;
#         }
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "movement_enabled" {
#         DATATYPE  H5T_ENUM {
#             H5T_STD_I8LE;
#             "FALSE"            0;
#             "TRUE"             1;
#         }
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "success" {
#         DATATYPE  H5T_ENUM {
#             H5T_STD_I8LE;
#             "FALSE"            0;
#             "TRUE"             1;
#         }
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
# }
# DATASET "episode_info" {
#     DATATYPE  H5T_COMPOUND {
#         H5T_STRING {
#             STRSIZE 100;
#             STRPAD H5T_STR_NULLPAD;
#             CSET H5T_CSET_ASCII;
#             CTYPE H5T_C_S1;
#         } "task";
#         H5T_ENUM {
#             H5T_STD_I8LE;
#             "FALSE"            0;
#             "TRUE"             1;
#         } "is_success";
#         H5T_STD_I32LE "episode_id";
#     }
#     DATASPACE  SIMPLE { ( 1 ) / ( 1 ) }
# }
# DATASET "hand_image" {
#     DATATYPE  H5T_STD_U8LE
#     DATASPACE  SIMPLE { ( 111, 640, 480, 3 ) / ( H5S_UNLIMITED, 640, 480, 3 ) }
# }
# DATASET "image" {
#     DATATYPE  H5T_STD_U8LE
#     DATASPACE  SIMPLE { ( 111, 640, 480, 3 ) / ( H5S_UNLIMITED, 640, 480, 3 ) }
# }
# DATASET "robot_state" {
#     DATATYPE  H5T_IEEE_F64LE
#     DATASPACE  SIMPLE { ( 111, 15 ) / ( H5S_UNLIMITED, 15 ) }
# }
# GROUP "timestamps" {
#     DATASET "control_start" {
#         DATATYPE  H5T_STD_I64LE
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "policy_start" {
#         DATATYPE  H5T_STD_I64LE
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "sleep_start" {
#         DATATYPE  H5T_STD_I64LE
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "step_end" {
#         DATATYPE  H5T_STD_I64LE
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
#     DATASET "step_start" {
#         DATATYPE  H5T_STD_I64LE
#         DATASPACE  SIMPLE { ( 111 ) / ( H5S_UNLIMITED ) }
#     }
# }
# }
# }

In [2]:
# def create_video_file(suffix=".mp4", byte_contents=None):
#     # Create Temporary File #
#     temp_file = tempfile.NamedTemporaryFile(suffix=suffix)
#     filename = temp_file.name

#     # If Byte Contents Provided, Write To File #
#     if byte_contents is not None:
#         with open(filename, "wb") as binary_file:
#             binary_file.write(byte_contents)
#     return filename

# converts images from ndarray into mp4 video
def convert_to_video(images, save_path, fps):
    # Create aj for testing, dimension (13, 222, 356, 3)
    aj = np.array(images)
    # for i in range(aj.shape[0]):
    #     cv2.putText(aj[i, :, :], str(i+1), (aj.shape[2]//2-50*len(str(i+1)), aj.shape[1]//2+50), cv2.FONT_HERSHEY_DUPLEX, 5, (255, 30, 30), 10)  # Blue number

    # size = 720 * 16 // 9, 720
    # duration = 2
    # fps = 15
    out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (aj.shape[2], aj.shape[1]))
    for i in range(aj.shape[0]):
        data = aj[i, :, :, :]
        out.write(data)
    out.release()
    # with imageio.mimwrite(save_path, fps=fps) as vidwriter:
    #     for frame in images:
    #         vidwriter.append_data(frame)
           
# save dictionary to csvs   
def save_to_csv(obs_dict, path):
    for key, value in obs_dict.items():
        if isinstance(value, dict): # if value is dictionary, it needs to be merged first and then saved 
            dataframe = pd.DataFrame(value) # flatten into dataframe with pandas
            dataframe.to_csv(os.path.join(path, f"{key}.csv"), index=False)
        elif isinstance(value, (list, np.ndarray)): # turn list or np array directly to csv file
            if isinstance(value, np.ndarray):
                value = value.tolist() # turn into list
            dataframe = pd.DataFrame(value)
            dataframe.to_csv(os.path.join(path, f"{key}.csv"), index=False)
        else:
            # if isinstance(value, np.ndarray) and value.ndim == 1:  # Handle 1D arrays
            #     dataframe = pd.DataFrame(value)
            dataframe = pd.DataFrame([value])
            dataframe.to_csv(os.path.join(path, f"{key}.csv"), index=False)

# def get_hdf5_length(hdf5_file, keys_to_ignore=[]):
#     length = None

#     for key in hdf5_file.keys():
#         if key in keys_to_ignore:
#             continue

#         curr_data = hdf5_file[key]
#         if isinstance(curr_data, h5py.Group):
#             curr_length = get_hdf5_length(curr_data, keys_to_ignore=keys_to_ignore)
#         elif isinstance(curr_data, h5py.Dataset):
#             curr_length = len(curr_data)
#         else:
#             raise ValueError

#         if length is None:
#             length = curr_length
#         assert curr_length == length

#     return length

# convert hdf5 back into a nested dictionary
def load_hdf5_to_dict(hdf5_file, keys_to_ignore=[]):
    data_dict = {}

    print(list(hdf5_file.keys()))

    for key in hdf5_file.keys():
        if key in keys_to_ignore:
            continue

        curr_data = hdf5_file[key]
        if isinstance(curr_data, h5py.Group): # ungroup nested hdf5 groups
            data_dict[key] = load_hdf5_to_dict(curr_data, keys_to_ignore=keys_to_ignore)
        elif isinstance(curr_data, h5py.Dataset):
            data_dict[key] = np.array(curr_data)
        else:
            raise ValueError

    return data_dict

In [3]:
def trajectory_process(obs_file, output_dir):
    # make output_dir if not existing
    os.makedirs(output_dir, exist_ok = True)
    
    # extract dictionary from the hdf5 file
    obs_dict = load_hdf5_to_dict(obs_file)
    
    # first process images from the dictionary into videos
    fps = 30 # video fps constant
    
    if "image" in obs_dict.keys(): # write primary images to video
        primary_img = obs_dict["image"]
        save_path = os.path.join(output_dir, "primary_cam.mp4")
        convert_to_video(primary_img, save_path, fps=fps)
        
    if "wrist_image" in obs_dict.keys(): # write wrist images to video
        wrist_img = obs_dict["wrist_image"]
        save_path = os.path.join(output_dir, "wrist_cam.mp4")
        convert_to_video(wrist_img, save_path, fps = fps)
    
    # process action and robot_state data into csv files
    # if "task" in obs_dict.keys(): # print the task string
    #     print(obs_dict["task"][0]) # task is the same throughout the whole dictionary
    
    # create dictionary out of remaining values and write to csv files
    # should create 4 csv files: one for robot_state, action, controller_info, and timestamps
    other_obs = {key: value for key, value in obs_dict.items() if key not in ["image", "wrist_image", "task"]}
    
    # print(other_obs)
    
    save_to_csv(other_obs, output_dir)
    

In [13]:
hdf5_file = h5py.File("/home/demoaccount/Data/demoaccount2/teleop_data_collect/ur10e_tele/outputs/session_1/episode_0.h5", 'r') # change this to the correct filename

# print(hdf5_file.keys())

# print("action data: {}".format(hdf5_file['action']))
# print("action data attributes: {}".format(list(hdf5_file['action'].attrs)))

trajectory_process(hdf5_file, output_dir="results/session_1/episode_0") # change output_dir to the correct name

print("Done processing.")

# print(load_hdf5_to_dict(hdf5_file))

['action', 'controller_info', 'episode_info', 'image', 'robot_state', 'timestamps', 'wrist_image']
['controller_on', 'failure', 'movement_enabled', 'success']
['control_start', 'policy_start', 'sleep_start', 'step_end', 'step_start']
Done processing.


### MISC

In [17]:
hdf5_file = h5py.File("//home/demoaccount/Data/demoaccount2/teleop_data_collect/ur10e_tele/outputs/stuffed_toy/session_0/episode_1.h5", 'r+')

print(hdf5_file.keys())
print(list(hdf5_file['episode_info']))

task = "place the pink stuffed animal in the box"

dtype = np.dtype([('task', 'S100')])

data = task.encode('utf-8')

# print(data)

hdf5_file['episode_info']['task'] = data

# hdf5_file.create_dataset('episode_info', data=[(task.encode('utf-8'))], dtype=dtype)

print(hdf5_file["episode_info"])
print(list(hdf5_file['episode_info']))

hdf5_file.close()


<KeysViewHDF5 ['action', 'controller_info', 'episode_info', 'image', 'robot_state', 'timestamps', 'wrist_image']>
[(b'place the stuffy in the box', True, 1)]
<HDF5 dataset "episode_info": shape (1,), type "|V105">
[(b'place the pink stuffed animal in the box', True, 1)]


In [5]:

# class TrajectoryReader:
#     def __init__(self, filepath, read_images=True):
#         self._hdf5_file = h5py.File(filepath, "r")
#         # is_video_folder = "observations/videos" in self._hdf5_file
#         self._read_images = read_images # and is_video_folder
#         self._length = get_hdf5_length(self._hdf5_file)
#         self._video_readers = {}
#         self._index = 0

#     def length(self):
#         return self._length

#     def read_timestep(self, index=None, keys_to_ignore=[]):
#         # Make Sure We Read Within Range #
#         if index is None:
#             index = self._index
#         else:
#             assert not self._read_images
#             self._index = index
#         assert index < self._length

#         # Load Low Dimensional Data #
#         keys_to_ignore = [*keys_to_ignore.copy(), "videos"]
#         timestep = load_hdf5_to_dict(self._hdf5_file, self._index, keys_to_ignore=keys_to_ignore)

#         # Load High Dimensional Data #
#         if self._read_images:
#             camera_obs = self._uncompress_images()
#             timestep["observations"]["image"] = camera_obs

#         # Increment Read Index #
#         self._index += 1

#         # Return Timestep #
#         return timestep

#     def _uncompress_images(self):
#         # WARNING: THIS FUNCTION HAS NOT BEEN TESTED. UNDEFINED BEHAVIOR FOR FAILED READING. #
#         video_folder = self._hdf5_file["observations/videos"]
#         camera_obs = {}

#         for video_id in video_folder:
#             # Create Video Reader If One Hasn't Been Made #
#             if video_id not in self._video_readers:
#                 serialized_video = video_folder[video_id]
#                 filename = create_video_file(byte_contents=serialized_video)
#                 self._video_readers[video_id] = imageio.get_reader(filename)

#             # Read Next Frame #
#             camera_obs[video_id] = yield self._video_readers[video_id]
#             # Future Note: Could Make Thread For Each Image Reader

#         # Return Camera Observation #
#         return camera_obs

#     def close(self):
#         self._hdf5_file.close()